In [1]:
%run load_pga_tee_time.py
df_tee_time = df

   Group Number            Tee Time  Start Tee Group Location  Group Hole  \
0             1 2025-03-06 12:40:00          1    Coming soon           1   
1             1 2025-03-06 12:40:00          1    Coming soon           1   

                             Course Name Group Status      Player Name Country  
0  Arnold Palmer's Bay Hill Club & Lodge  NOT_STARTED   Brian Campbell     USA  
1  Arnold Palmer's Bay Hill Club & Lodge  NOT_STARTED  Jacob Bridgeman     USA  


In [2]:
import pandas as pd

# User input for the date
date_input = '2025-03-06'

# Convert user input to a pandas Timestamp
date_input = pd.to_datetime(date_input)

# Filter the DataFrame to include only rows with the specified date
df_tee_time = df_tee_time[df_tee_time['Tee Time'].dt.date == date_input.date()]

print(df_tee_time)


    Group Number            Tee Time  Start Tee Group Location  Group Hole  \
0              1 2025-03-06 12:40:00          1    Coming soon           1   
1              1 2025-03-06 12:40:00          1    Coming soon           1   
2              2 2025-03-06 12:50:00          1    Coming soon           1   
3              2 2025-03-06 12:50:00          1    Coming soon           1   
4              3 2025-03-06 13:00:00          1    Coming soon           1   
..           ...                 ...        ...            ...         ...   
67            34 2025-03-06 18:30:00          1    Coming soon           1   
68            35 2025-03-06 18:40:00          1    Coming soon           1   
69            35 2025-03-06 18:40:00          1    Coming soon           1   
70            36 2025-03-06 18:50:00          1    Coming soon           1   
71            36 2025-03-06 18:50:00          1    Coming soon           1   

                              Course Name Group Status      Pla

In [3]:
%run load_pga_tournament.py
df_tournament = df

  POS        PLAYER TOT     THRU R1 R2 R3 R4  total_stroke
0   -   B. Campbell   E  7:40 AM  -  -  -  -             0
1   -  J. Bridgeman   E  7:40 AM  -  -  -  -             0


C:\Users\rypi3\source\repos\data_golf\load_pga_tournament.py:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('-', inplace=True)


In [4]:
%run load_pga_fedexcup_rank.py
df_rank = df

    Player Name  Country Projected Rank Official Rank Point Data Projected  \
0  Ludvig Åberg   Sweden              1             1            1,678.417   
1   Sepp Straka  Austria              2             2            1,612.595   

  Point Data Official  
0                 978  
1                 913  


In [5]:
# Function to extract last name
def extract_last_name(full_name):
    return full_name.split()[-1]

In [6]:
def extract_name_with_initial(full_name):
    name_parts = full_name.split()
    first_initial = name_parts[0][0] + '.' if name_parts else ''
    last_name = name_parts[-1] if len(name_parts) > 1 else ''
    return f"{first_initial} {last_name}"

In [7]:
# Apply the function to the Full Name column to create a new Last Name column
df_tournament['Name_Key'] = df_tournament['PLAYER'].apply(extract_last_name)
df_tee_time['Name_Key'] = df_tee_time['Player Name'].apply(extract_last_name)
df_rank['Name_Key'] = df_rank['Player Name'].apply(extract_last_name)

C:\Users\rypi3\AppData\Local\Temp\ipykernel_20416\4203428155.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tee_time['Name_Key'] = df_tee_time['Player Name'].apply(extract_last_name)


In [8]:
# Apply the function to the Full Name column to create a new Last Name column
df_tournament['Name_Key2'] = df_tournament['PLAYER'].apply(extract_name_with_initial)
df_tee_time['Name_Key2'] = df_tee_time['Player Name'].apply(extract_name_with_initial)
df_rank['Name_Key2'] = df_rank['Player Name'].apply(extract_name_with_initial)

C:\Users\rypi3\AppData\Local\Temp\ipykernel_20416\643551268.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tee_time['Name_Key2'] = df_tee_time['Player Name'].apply(extract_name_with_initial)


In [9]:
df_tee_time.tail(2)

,Group Number,Tee Time,Start Tee,Group Location,Group Hole,Course Name,Group Status,Player Name,Country,Name_Key,Name_Key2
70,36,2025-03-06 18:50:00,1,Coming soon,1,Arnold Palmer's Bay Hill Club & Lodge,NOT_STARTED,Harris English,USA,English,H. English
71,36,2025-03-06 18:50:00,1,Coming soon,1,Arnold Palmer's Bay Hill Club & Lodge,NOT_STARTED,Rafael Campos,PUR,Campos,R. Campos


In [10]:
df_times_rank = pd.merge(df_tee_time, df_tournament, on='Name_Key2', how='left')
df_times_rank = pd.merge(df_times_rank, df_rank, on='Name_Key2', how='left')

In [11]:
print(df_rank.shape[1] + df_tee_time.shape[1] + df_tournament.shape[1])

30


In [12]:
df_times_rank = df_times_rank[['Group Number','Tee Time', 'Player Name_x','POS','Official Rank']]

In [13]:
# Convert numeric columns to integer
df_times_rank['Official Rank'] = df_times_rank['Official Rank'].fillna(1000).astype(int)

In [14]:
df_times_rank.dtypes

Group Number              int64
Tee Time         datetime64[ns]
Player Name_x            object
POS                      object
Official Rank             int64
dtype: object

In [15]:
# Group by the key and calculate the difference between rows
df_times_rank['Difference'] = df_times_rank.groupby('Group Number')['Official Rank'].diff().fillna(0)

In [16]:
print(f'original row count: {df_times_rank.shape[0]}')

original row count: 73


In [17]:
# Remove duplicates from upcoming tee times
df_times_rank.drop_duplicates(inplace=True)

In [18]:
print(f'original row count: {df_times_rank.shape[0]}')


original row count: 73


In [19]:
df_best_groups = df_times_rank[df_times_rank['Difference']>100]
df_best_groups.head()

,Group Number,Tee Time,Player Name_x,POS,Official Rank,Difference
28,14,2025-03-06 14:55:00,Viktor Hovland,-,121,118.0
38,19,2025-03-06 15:50:00,Jackson Koivun,-,1000,947.0
40,20,2025-03-06 16:00:00,Mackenzie Hughes,-,127,120.0
72,36,2025-03-06 18:50:00,Rafael Campos,-,151,142.0


In [20]:
# Filter rows in df_other based on Group Numbers in df_best_groups
df_bet = df_times_rank[df_times_rank['Group Number'].isin(df_best_groups['Group Number'])]

df_bet.head(25)


,Group Number,Tee Time,Player Name_x,POS,Official Rank,Difference
27,14,2025-03-06 14:55:00,Hideki Matsuyama,-,3,0.0
28,14,2025-03-06 14:55:00,Viktor Hovland,-,121,118.0
37,19,2025-03-06 15:50:00,Isaiah Salinda,-,53,0.0
38,19,2025-03-06 15:50:00,Jackson Koivun,-,1000,947.0
39,20,2025-03-06 16:00:00,Maverick McNealy,-,7,0.0
40,20,2025-03-06 16:00:00,Mackenzie Hughes,-,127,120.0
71,36,2025-03-06 18:50:00,Harris English,-,9,0.0
72,36,2025-03-06 18:50:00,Rafael Campos,-,151,142.0


In [21]:
df_bet[df_bet['Difference'] == 0]

,Group Number,Tee Time,Player Name_x,POS,Official Rank,Difference
27,14,2025-03-06 14:55:00,Hideki Matsuyama,-,3,0.0
37,19,2025-03-06 15:50:00,Isaiah Salinda,-,53,0.0
39,20,2025-03-06 16:00:00,Maverick McNealy,-,7,0.0
71,36,2025-03-06 18:50:00,Harris English,-,9,0.0


In [25]:
from tabulate import tabulate

ModuleNotFoundError: No module named 'tabulate'